In [2]:
import pandas as pd

ham_df = pd.read_csv('./datasets/ham_.csv')
phishing_df = pd.read_csv('./datasets/phishing_.csv')
df = ham_df + phishing_df

In [ ]:
# for col in ham_df.columns:
#     print(col, ham_df[col].unique())
# print()
# for col in phishing_df.columns:
#     print(col, phishing_df[col].unique())

In [4]:
ham_df['Phishy'] = False

In [5]:
df = pd.concat([phishing_df,ham_df],axis=0)

In [6]:
del df['Unnamed: 0']
del df['Html iFrame']
del df['HTML content']

#### Data encoding

In [7]:
from sklearn.preprocessing import LabelEncoder

for col in df.columns:
    if set(df[col].unique()).issubset({True,False}):
        df[col] = LabelEncoder().fit_transform(df[col])

In [8]:
del df['Encoding']

In [62]:
df.iloc[:,0]

0        0
1        0
2        0
3        0
4        0
        ..
10700    0
10701    0
10702    0
10703    0
10704    0
Name: Html Form, Length: 12100, dtype: int64

In [19]:
from sklearn.preprocessing import MinMaxScaler

df = pd.DataFrame(MinMaxScaler().fit_transform(df),columns=df.columns)


In [20]:
df

,Html Form,Attachments,Flash content,URLs,External Resources,Javascript,Css,IPs in URLs,@ in URLs,Phishy
0,0.0,0.0,0.0,0.055300,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.004608,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.004608,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.004608,0.0,0.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.078341,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
21401,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
21402,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
21403,0.0,0.0,0.0,0.002304,0.0,0.0,0.0,0.0,0.0,0.0
21404,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], stratify=df.iloc[:,-1], test_size=0.3, random_state=42)

In [24]:
import numpy as np

print(np.bincount(y_train))
print(np.bincount(y_test))

[7493 7491]
[3212 3210]


In [28]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb = GaussianNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6281532232949237


In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8763625038928683


In [32]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score



svm = SVC(kernel='linear')

svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6628776082217378


In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy2 = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy2)


Accuracy: 0.8766739333540953


In [38]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier



dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

y_pred = dt_classifier.predict(X_test)

accuracy3 = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy3)


Accuracy: 0.8763625038928683


In [41]:
from sklearn.linear_model import LogisticRegression



logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.674400498287138


In [42]:
import xgboost as xgb



dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "multi:softmax",
    "num_class": 3,
    "max_depth": 3,
    "eta": 0.1,
    "gamma": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "eval_metric": "merror",
    "seed": 42
}

num_rounds = 100
xgb_model = xgb.train(params, dtrain, num_rounds)

y_pred = xgb_model.predict(dtest)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8763625038928683
